In [20]:
import numpy as np
import pandas as pd

In [21]:
df = pd.read_csv("sp500_testdata_2.csv")

In [22]:
df.Date = pd.to_datetime(df.Date)
df["TR"] = ""
df.TR.loc[0] = df.high.loc[0] - df.low.loc[0]    # Initial value for TR
df.TR.loc[1:] = ""
df["N"] = ""
df["N"].loc[0] = df.high.loc[0] - df.low.loc[0]   # Initial value for N
df["N"].loc[1:] = ""
df["NPP"] = ""
df.NPP.loc[0] = 0.00     # Initial value for NPP
df["FL"] = ""
df["SL"] = ""
df.FL.loc[0] = 0.00      # Initial value for FL
df.SL.loc[0] = 0.00      # Initial value for SL
df["Sep"] = ""           # Percentage separation between FL and SL
df.Sep.loc[0] = 0.00
df["TrendDir"] = ""
df["TrendDir"] = df["TrendDir"].astype('str', errors='ignore')
df["TrendDay"] = ""
df["TrendDay"] = df["TrendDay"].astype('int', errors='ignore')
df["FE_FL"] = ""
# df.FE_FL.round(3)       
df["AE_FL"] = ""
df["SP"] = ""
df["En_price1"] = ""
df["Position"] = ""
df["Counter1"] = ""
df["Counter2"] = ""
df["SDEE"] = ""   # same day entry/exit: Position=0 or 1 and trade entry/exit occurs on same day due to SL (a & b) or TP (c) or RE (d)     
df["a"] = ""
df["b"] = ""
df["c"] = ""
df["d"] = ""
df["RE_CAN"] = ""
df["RE_CAN"] = df["RE_CAN"].astype('bool', errors='ignore')
# df["EnP2_RE"] = ""
df["En_price2"] = ""
df["Ex_price1"] = ""
df["Ex_price2"] = ""
df["Ex_price3"] = ""
df["Ex_price4"] = ""
df["Ex_price5"] = ""
df["P_L"] = ""
df["Capital"] = ""
df["Capital"].loc[0:25] = 25000

df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]] = df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]].apply(pd.to_numeric)



C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
df.dtypes

Date         datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
TR                  float64
N                   float64
NPP                 float64
FL                  float64
SL                  float64
Sep                 float64
TrendDir             object
TrendDay            float64
FE_FL               float64
AE_FL               float64
SP                  float64
En_price1           float64
Position            float64
Counter1             object
Counter2             object
SDEE                 object
a                   float64
b                   float64
c                   float64
d                   float64
RE_CAN                 bool
En_price2           float64
Ex_price1           float64
Ex_price2           float64
Ex_price3           float64
Ex_price4           float64
Ex_price5           float64
P_L                 float64
Capital             float64
dtype: object

In [24]:
#  3 loops: 
#   a) calculate True Range for every row after the first row
#   b) calculate N for every row after the first row
#   c) calculate NPP for every row after first row


# a)
#  True Range = max( absolute value(H - L), absolute value (H - C(previous)), absoulute value (C(previous) - L))
rownum = 1
for rownum in range(1,1500):
    df.TR.loc[rownum:] = max(abs(df.high.loc[rownum] - df.low.loc[rownum]), abs(df.high.loc[rownum] - df.close.loc[rownum-1]), abs(df.close.loc[rownum-1] - df.low.loc[rownum]))
    rownum += 1
    
#  b)
#  N = N(previous) + (TR-N(previous)) / (0.5 * 20 + 1)
Constant = 20
rownum = 1
for rownum in range(1, 1500):
    df.N.loc[rownum:] = df.N.loc[rownum - 1]+(df.TR.loc[rownum]-df.N.loc[rownum - 1])/(0.5 * (Constant + 1))
    rownum +=1
    
# c)
df.NPP.loc[rownum:].round(3)
rownum = 1
for rownum in range(1, 1500):
    df.NPP.loc[rownum:] = df.N.loc[rownum]/df.close.loc[rownum-1] * 100
    rownum += 1
    
# try replacing this with a List Comprehension    
# [df.NPP.loc[rownum:] for df.NPP.loc[rownum] in range(1.599) if df.NPP.loc[rownum] is float]


C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
# 3 min 42 sec to finish execution on 1500 rows
# loop that yields FL, SL, Sep values after index position zero

rownum = 1
FL_period = 7
SL_period = 20
for rownum in range(1, 1500):
    df.FL.loc[rownum:] = df.FL.loc[rownum-1] + (df.close.loc[rownum] - df.FL.loc[rownum-1])/((FL_period+1)/2)
    df.SL.loc[rownum:] = df.SL.loc[rownum-1] + (df.close.loc[rownum] - df.SL.loc[rownum-1])/((SL_period+1)/2)
    df.Sep.loc[rownum] = (df.FL.loc[rownum] - df.SL.loc[rownum])/df.SL.loc[rownum] * 100
    rownum += 1
    
    
    
# Determining TrendDir based on boolean result for question: FL > SL ?
# start at index = 25 to give ramp-up time for lags
rownum = 100
for rownum in range(25, 1500):
    if df.FL.loc[rownum] > df.SL.loc[rownum]:
        df.TrendDir.loc[rownum] = "Up"
    else:    
        df.TrendDir.loc[rownum] = "Down"
rownum += 1

# iterate through rows starting at row 101 to find where trends initiate (TrendDay=0) and continue (TrendDay>0)

rownum = 101           # start at index = 101 (TrendDay at index=100 is set to 0)
for rownum in range(25, 1500):
    if df.TrendDir.loc[rownum] == df.TrendDir.loc[rownum-1]:
        df.TrendDay.loc[rownum] = (df.TrendDay.loc[rownum-1] + 1)
    else:
        df.TrendDay.loc[rownum] = 0
rownum += 1


C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
# 1 min 23 sec to finish execution on 1500 rows

""" FE_FL and AE_FL - favorable & adverse distance of a day's High & Low from the previous day's FL in terms of previous N """

rownum = 25   # start after series for TrendDay transitions from blank (string value) to int
for rownum in range(25, 1500):
    if df.TrendDay.loc[rownum] == 0:        
        df.FE_FL.loc[rownum] = ""
        df.AE_FL.loc[rownum] = ""
    else:
        if df.TrendDir.loc[rownum] == "Up":
        
            #Favorable Excursion from FL in terms of N for long position:
            if df.high.loc[rownum] < df.FL.loc[rownum-1]:
                df.FE_FL.loc[rownum] = 0
            else:
                df.FE_FL.loc[rownum] = (df.high.loc[rownum] - df.FL.loc[rownum-1])/df.N.loc[rownum-1]

            #Adverse Excursion from FL in terms of N for for long position (display positive value):
            if df.low.loc[rownum] > df.FL.loc[rownum-1]:
                df.AE_FL.loc[rownum] = 0
            else:
                df.AE_FL.loc[rownum] = abs((df.low.loc[rownum] - df.FL.loc[rownum-1])/df.N.loc[rownum-1])
    
        elif df.TrendDir.loc[rownum] == "Down":
        
            #Favorable Excursion from FL in terms of N for Short position:
            if df.low.loc[rownum] > df.FL.loc[rownum-1]:
                df.FE_FL.loc[rownum] = 0
            else:
                df.FE_FL.loc[rownum] = (df.FL.loc[rownum-1] - df.low.loc[rownum])/df.N.loc[rownum-1]

            #Adverse Excursion from FL in terms of N for for Short position (display positive value):
            if df.high.loc[rownum] < df.FL.loc[rownum-1]:
                df.AE_FL.loc[rownum] = 0
            else:
                df.AE_FL.loc[rownum] = abs((df.FL.loc[rownum-1] - df.high.loc[rownum])/df.N.loc[rownum-1])        
            
# incrememnt to next bar
rownum += 1

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [27]:
df["SP"].fillna("", inplace = True)
df["En_price1"].fillna("", inplace = True)
df["Position"].fillna("", inplace = True)
df["Counter1"].fillna("", inplace = True)
df["Counter2"].fillna("", inplace = True)
df["SDEE"].fillna("", inplace = True)
df["a"].fillna("", inplace = True)
df["b"].fillna("", inplace = True)
df["c"].fillna("", inplace = True)
df["d"].fillna("", inplace = True)
df["RE_CAN"].fillna("", inplace = True)
df["En_price2"].fillna("", inplace = True)
df["Ex_price1"].fillna("", inplace = True)
df["Ex_price2"].fillna("", inplace = True)
df["Ex_price3"].fillna("", inplace = True)
df["Ex_price4"].fillna("", inplace = True)
df["Ex_price5"].fillna("", inplace = True)
df["P_L"].fillna("", inplace = True)
df["Capital"].fillna("", inplace = True)


In [28]:
"""  calculates and displays in Dataframe A,B,C volatility levels """


addpos = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},                            # syntax to call, enter script as follows:
          "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}                             # addpos["L"]["2"]  yields: 0.015
          }                                                                        # stoploss["S"]["3"]  yields: 0.0195

stoploss = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

takeprof = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

re_entry = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

X = 0.25   # General Risk Mgmt. multiplier

rownum = 10
for rownum in range (1, 1500):
    if df.TrendDay.loc[rownum] > 0 and df.TrendDir.loc[rownum] == "Up":

        """ AP_U function: AE_FL: price at which trigger AP """
        if df.NPP.loc[rownum - 1] < addpos["L"]["1"]:
            a = df.FL.loc[rownum - 1] +  X * df.N.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] <= addpos["L"]["2"]:                    
            a = df.FL.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] <= addpos["L"]["3"]:
            a = df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]
        else:
            a = df.FL.loc[rownum - 1] - 6.4 * X * df.N.loc[rownum - 1]

        df.a.loc[rownum] = a   # print value in dataframe slice 

        """ SL_U function: AE_FL: price at which triggers SL """
        if df.NPP.loc[rownum - 1] <= stoploss["L"]["1"]:
            b =  df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]     # stop loss when price retraces 1N below FL
        elif df.NPP.loc[rownum - 1] <= stoploss["L"]["2"]:
            b = df.FL.loc[rownum - 1] - 6 * X * df.N.loc[rownum - 1]  # stop loss when price retraces 150%N below FL
        elif df.NPP.loc[rownum - 1] <= stoploss["L"]["3"]:
            b = df.FL.loc[rownum - 1] - 7 * X * df.N.loc[rownum - 1]   # stop loss when price retraces  175%N below FL
        else:
            b =  df.FL.loc[rownum - 1] - 8 * X * df.N.loc[rownum - 1]

        df.b.loc[rownum] = b

        """ TP_U function: FE_FL: price at which triggers TP """
        if df.NPP.loc[rownum - 1] <= takeprof["L"]["1"]:
            c = df.FL.loc[rownum - 1] + 5 * X * df.N.loc[rownum - 1]   # for NPP < 1.10%: take profit when price exceeds 125%N above previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["L"]["2"]:
            c = df.FL.loc[rownum - 1] + 5.6 * X * df.N.loc[rownum - 1]    # for NPP < 1.50%: take profit when price exceeds 140%N (5.6 * X) above previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["L"]["3"]:
            c = df.FL.loc[rownum - 1] + 7 * X * df.N.loc[rownum - 1]    # for NPP < 1.95%: take profit when price exceeds 175%N (7*X*N) above previous FL
        else:
            c = df.FL.loc[rownum - 1] + 7.5 * X * df.N.loc[rownum - 1]                    

        df.c.loc[rownum] = c

        rownum += 1
        
    elif df.TrendDay.loc[rownum] > 0 and df.TrendDir.loc[rownum] == "Down":
        
        """ AP_D function: AE_FL: price at which triggers AP """
        if df.NPP.loc[rownum - 1] < addpos["S"]["1"]:
            a = df.FL.loc[rownum - 1] -  X * df.N.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] <= addpos["S"]["2"]:                    
            a = df.FL.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] <= addpos["S"]["3"]:
            a = df.FL.loc[rownum - 1] + 5 * X * df.N.loc[rownum - 1]
        else:
            a = df.FL.loc[rownum - 1] + 6.4 * X * df.N.loc[rownum - 1]

        df.a.loc[rownum] = a   # print value in dataframe slice  
        
        """ SL_D function: AE_FL: price at which trigger SL """
        if df.NPP.loc[rownum - 1] <= stoploss["S"]["1"]:
            b =  df.FL.loc[rownum - 1] + 5 * X * df.N.loc[rownum - 1]     # stop loss when price retraces 1N above FL
        elif df.NPP.loc[rownum - 1] <= stoploss["S"]["2"]:
            b = df.FL.loc[rownum - 1] + 6 * X * df.N.loc[rownum - 1]  # stop loss when price retraces 150%N above FL
        elif df.NPP.loc[rownum - 1] <= stoploss["S"]["3"]:
            b = df.FL.loc[rownum - 1] + 7 * X * df.N.loc[rownum - 1]   # stop loss when price retraces  175%N above FL
        else:
            b =  df.FL.loc[rownum - 1] + 8 * X * df.N.loc[rownum - 1]

        df.b.loc[rownum] = b        
        
        
        """ TP_D function: FE_FL: price at which triggers TP """
        if df.NPP.loc[rownum - 1] <= takeprof["S"]["1"]:
            c = df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]   # for NPP < 1.10%: take profit when price exceeds 125%N below previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["S"]["2"]:
            c = df.FL.loc[rownum - 1] - 5.6 * X * df.N.loc[rownum - 1]    # for NPP < 1.50%: take profit when price exceeds 140%N (5.6 * X) below previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["S"]["3"]:
            c = df.FL.loc[rownum - 1] - 7 * X * df.N.loc[rownum - 1]    # for NPP < 1.95%: take profit when price exceeds 175%N (7*X*N) below previous FL
        else:
            c = df.FL.loc[rownum - 1] - 7.5 * X * df.N.loc[rownum - 1]                    

        df.c.loc[rownum] = c

        rownum += 1
                
        


C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [30]:
df.iloc[274:330, 2:22].round({'N': 2, 'NPP': 3, 'FL': 2, 'SL': 2, 'Sep': 3, 'FE_FL': 3, 'AE_FL': 3, 'TrendDay': 0, 'SP': 2, 'a':0, 'b':0, 'c':0})

,high,low,close,TR,N,NPP,FL,SL,Sep,TrendDir,TrendDay,FE_FL,AE_FL,SP,En_price1,Position,Counter1,Counter2,SDEE,a
274,721.75,670.00,707.25,51.75,53.26,7.891,673.06,675.79,-0.403,Down,67.0,0,1.12473,,,,,,,747.14
275,718.75,687.00,692.00,31.75,51.22,7.241,677.79,677.33,0.069,Up,0.0,,,718.75,,0,,,,
276,711.25,687.00,698.25,24.25,48.65,7.030,682.91,679.32,0.528,Up,1.0,0.65323,0,711.25,,,,,,595.85
277,707.50,670.75,677.00,36.75,47.51,6.805,681.43,679.10,0.343,Up,2.0,0.505508,0.249933,707.5,,,,,,605.073
278,689.75,631.50,688.50,58.25,48.54,7.169,683.20,680.00,0.471,Up,3.0,0.175077,1.05088,689.75,,,,,,605.409
279,696.50,659.75,675.75,36.75,47.41,6.887,681.34,679.59,0.257,Up,4.0,0.274051,0.483111,,690,1,1,,,605.54
280,718.50,672.00,716.25,46.50,47.33,7.004,690.06,683.08,1.022,Up,5.0,0.783811,0.196912,,,,,,,605.474
281,721.75,690.25,706.50,31.50,45.82,6.397,694.17,685.31,1.293,Up,6.0,0.669496,0,,,,,,,614.342
282,715.00,677.00,696.00,38.00,45.07,6.380,694.63,686.33,1.209,Up,7.0,0.45453,0.374809,,,,,,,620.862
283,707.00,680.00,684.75,27.00,43.35,6.229,692.16,686.18,0.871,Up,8.0,0.274428,0.324575,,,,,,,622.51


In [29]:
"""  Identifies all instances of Long & Short entries:  sets and displays in Dataframe: En_price1, Position = 1, Counter1 = 1  """

tick = 0.25
tick_val = 12.50
Capital = float(25000)

rownum = 50                                                 # start looking for uptrend starts from row 50


for rownum in range(1, 1499):
            
    if df.TrendDay.loc[rownum] == 0 and df.TrendDir.loc[rownum] == "Up":          
        Position = 0
        df.Position.loc[rownum] = 0
        SP = df.high.loc[rownum]                            # assign the 'high' slice to the SP variable
        df.SP.loc[rownum] = SP                              # equate the SP slice with the SP variable
        rownum += 1
        
        while df.TrendDay.loc[rownum] > 0:              # this 'while' loop finds initial entry, records initial P_L, then advances to next day
            if df.high.loc[rownum] > SP:
                En_price1 = SP + tick
                df.En_price1.loc[rownum] = En_price1
                Counter1 = 1
                df.Counter1.loc[rownum] = 1
                Position = 1                                # 'position' variable
                df.Position.loc[rownum] = Position         # this [rownum] dataframe slice equal to this [rownum] value for position variable
                df.P_L.loc[rownum] = df.close.loc[rownum] - En_price1        # yes, you can add a value contained in dataframe slice with a local variable that's a float!
                rownum += 1
                break
            else:
                SP = df.high.loc[rownum]
                df.SP.loc[rownum] = SP
                rownum += 1  
    
    elif df.TrendDay.loc[rownum] == 0 and df.TrendDir.loc[rownum] == "Down":
        Position = 0
        df.Position.loc[rownum] = 0
        SP = df.low.loc[rownum]
        df.SP.loc[rownum] = SP
        rownum += 1        

        while df.TrendDay.loc[rownum] > 0:
            if df.low.loc[rownum] < SP:
                En_price1 = SP - tick
                df.En_price1.loc[rownum] = En_price1
                Counter1 = 1
                df.Counter1.loc[rownum] = Counter1
                Position = 1
                df.Position.loc[rownum] = Position
                P_L = En_price1 - df.close.loc[rownum] 
                rownum += 1
                break
            else:
                SP = df.low.loc[rownum]
                df.SP.loc[rownum] = SP
                rownum += 1    

    
         

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [45]:
"""  
Iterates through rows after Counter1 = 1:  Assess each Pos=1,0,2 scenario against Volatility limits A,B,C,D  
                                            when necessary:
                                           - Set En_price1, En_price2
                                           - Set Ex_price1, Ex_price2, Ex_price3, Ex_price4
                                           - Update Counter(s)
                                           - Update Position(s)
                                           - Caculate & display P_L and change in Capital
                                           - move to next day (increment rownum)
"""

tick = 0.25
tick_val = 12.50
Capital = float(25000)

rownum = 50                                                 # start looking for uptrend starts from row 50


for rownum in range(50, 1499):
    
    if df.TrendDay.loc[rownum] >= 1 and df.TrendDir == "Up":
        
        
        """  CASE 1:  Position = 1 """
        if df.Position.loc[rownum] == 1:
            
            """ Identify Candle """
            if df.open.loc[rownum] > df.close.loc[rownum]:
                candle = "dark"
            elif df.open.loc[rownum] < df.close.loc[rownum]:
                candle = "open"
            elif df.open.loc[rownum] == df.close.loc[rownum]:
                candle_range = df.high.loc[rownum] - df.low.loc[rownum]
                if df.open.loc[rownum] >= (df.low.loc[rownum] + candle_range/2):   # open & close occur at middle or in top half of range
                    candle = "doji_open"
                else:
                    candle = "doji_dark"    # open & close occur in lower half of range            
                        
            if candle == "open":
                if df.low.loc[rownum] < b:        # SL_L: AE_FL add position AND stop loss on 2 positions
                    En_price2 = a - tick
                    Ex_price3 = b - tick
                    Ex_price4 = b - tick
                    Counter1 = ""
                    Counter2 = ""
                    Position = 0
                    df.Position.loc[rownum] = Position
                    df.P_L.loc[rownum] = [(En_price1 - Ex_price3) + (En_price1 - Ex_price4)] /tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum -1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.low.loc[rownum] < a:       # AP_L:  AE_FL add position and carry 2 to next day
                    En_price2 = a - tick
                    Counter1 += 1
                    Counter2 = 1
                    Position = 2
                    df.Position.loc[rownum] = Position
                    df.P_L.loc[rownum] = [(df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)] /tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.high.loc[rownum] > c:      # TP_L: FE_FL take profit on 1 position and stage for Re-Entry
                    Ex_price1 = c + tick
                    Counter1 = ""
                    Position = 0
                    df.Position.loc[rownum] = Position
                    RE_CAN = True
                    df.P_L.loc[rownum] = (Ex_price1 - En_price1)/tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]  
                    rownum += 1

                elif df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:    # FE_FL < c and AE_FL > a ---> No action, ride trend
                    Counter1 += 1
                    df.Position.loc[rownum] = df.Position.loc[rownum - 1]
                    df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1

            elif df.Position.loc[rownum - 1] == 1 and candle == "dark":

                if df.high.loc[rownum] > c:      # FE_FL take profit on 1 position and stage for Re-Entry
                    Ex_price1 = c + tick
                    Counter1 = ""
                    Position = 0
                    df.Position.loc[rownum] = Position
                    RE_CAN = True
                    df.P_L.loc[rownum] = (Ex_price1 - En_price1) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.low.loc[rownum] < b:        # AE_FL add position AND stop loss on 2 positions
                    En_price2 = a - tick
                    Ex_price3 = b - tick
                    Ex_price4 = b - tick
                    Counter1 = ""
                    Counter2 = ""
                    Position = 0
                    df.Position.loc[rownum] = Position
                    df.P_L.loc[rownum] = [(En_price1 - Ex_price3) + (En_price1 - Ex_price4)] / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum -1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.low.loc[rownum] < a:       # AE_FL add position and carry 2 to next day
                    En_price2 = a - tick
                    Counter1 += 1
                    Counter2 = 1
                    Position = 2
                    df.Position.loc[rownum] = Position
                    df.P_L.loc[rownum] = [(df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)] / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:    # FE_FL < c and AE_FL > a ---> No action, ride trend
                    Counter1 += 1
                    df.Position.loc[rownum] = df.Position.loc[rownum - 1]
                    df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1     
            else:
                rownum += 1
                    
                    
            
            
            
            
            
            
            
            
            
            

                
                
     
    
    
    
    elif df.TrendDay.loc[rownum] >= 1 and df.TrendDir.loc[rownum] == "Down":
        
        if df.Position.loc[rownum] == 1:
            
            """ Identify Candle """
            if df.open.loc[rownum] > df.close.loc[rownum]:
                candle = "dark"
            elif df.open.loc[rownum] < df.close.loc[rownum]:
                candle = "open"
            elif df.open.loc[rownum] == df.close.loc[rownum]:
                candle_range = df.high.loc[rownum] - df.low.loc[rownum]
                if df.open.loc[rownum] >= (df.low.loc[rownum] + candle_range/2):   # open & close occur at middle or in top half of range
                    candle = "doji_open"
                else:
                    candle = "doji_dark"    # open & close occur in lower half of range         
        
        
        
        Position = 0
        df.Position.loc[rownum] = 0
        SP = df.low.loc[rownum]
        df.SP.loc[rownum] = SP
        rownum += 1        

        while df.TrendDay.loc[rownum] > 0:
            if df.low.loc[rownum] < SP:
                En_price1 = SP - tick
                df.En_price1.loc[rownum] = En_price1
                Counter1 = 1
                df.Counter1.loc[rownum] = Counter1
                Position = 1
                df.Position.loc[rownum] = Position
                P_L = En_price1 - df.close.loc[rownum] 
                rownum += 1
                break
            else:
                SP = df.low.loc[rownum]
                df.SP.loc[rownum] = SP
                rownum += 1    

    else:
        rownum += 1
         

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [32]:
    
"""  LONG Cases:  Position = 1, 0, and 2   """

rownum = 10

while df.TrendDay.loc[rownum] > 0:
    if df.open.loc[rownum] > df.close.loc[rownum]:
        candle = "dark"
    elif df.open.loc[rownum] < df.close.loc[rownum]:
        candle = "open"
    elif df.open.loc[rownum] == df.close.loc[rownum]:
        candle_range = df.high.loc[rownum] - df.low.loc[rownum]
        if df.open.loc[rownum] >= (df.low.loc[rownum] + candle_range/2):   # open & close occur at middle or in top half of range
            candle = "doji_open"
        else:
            candle = "doji_dark"    # open & close occur in lower half of range
    

                
# Case 1:  Position = 1: check for  AP, SL, TP, or ride   
        
if df.TrendDay.loc[rownum] >= 1:
    
    
    if df.Position.loc[rownum - 1] == 1 and candle == "open":
                
        if df.low.loc[rownum] < b:        # AE_FL add position AND stop loss on 2 positions
            En_price2 = a - tick
            Ex_price3 = b - tick
            Ex_price4 = b - tick
            Counter1 = ""
            Counter2 = ""
            Position = 0
            df.Position.loc[rownum] = Position
            df.P_L.loc[rownum] = (En_price1 - Ex_price3) + (En_price1 - Ex_price4)
            df.Capital.loc[rownum] = df.Capital.loc[rownum -1] + df.P_L.loc[rownum]
            rownum += 1
                        
        elif df.low.loc[rownum] < a:       # AE_FL add position and carry 2 to next day
            En_price2 = a - tick
            Counter1 += 1
            Counter2 = 1
            Position = 2
            df.Position.loc[rownum] = Position
            df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1

        elif df.high.loc[rownum] > c:      # FE_FL take profit on 1 position and stage for Re-Entry
            Ex_price1 = c + tick
            Counter1 = ""
            Position = 0
            df.Position.loc[rownum] = Position
            RE_CAN = True
            df.P_L.loc[rownum] = (Ex_price1 - En_price1)
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]  
            rownum += 1

        elif df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:    # FE_FL < c and AE_FL > a ---> No action, ride trend
            Counter1 += 1
            df.Position.loc[rownum] = df.Position.loc[rownum - 1]
            df.P_L.loc[rownum] = df.close.loc[rownum] - En_price1
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1

    elif df.Position.loc[rownum - 1] == 1 and candle == "dark":

        if df.high.loc[rownum] > c:      # FE_FL take profit on 1 position and stage for Re-Entry
            Ex_price1 = c + tick
            Counter1 = ""
            Position = 0
            df.Position.loc[rownum] = Position
            RE_CAN = True
            df.P_L.loc[rownum] = (Ex_price1 - En_price1)
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1

        elif df.low.loc[rownum] < b:        # AE_FL add position AND stop loss on 2 positions
            En_price2 = a - tick
            Ex_price3 = b - tick
            Ex_price4 = b - tick
            Counter1 = ""
            Counter2 = ""
            Position = 0
            df.Position.loc[rownum] = Position
            df.P_L.loc[rownum] = (En_price1 - Ex_price3) + (En_price1 - Ex_price4)
            df.Capital.loc[rownum] = df.Capital.loc[rownum -1] + df.P_L.loc[rownum]
            rownum += 1

        elif df.low.loc[rownum] < a:       # AE_FL add position and carry 2 to next day
            En_price2 = a - tick
            Counter1 += 1
            Counter2 = 1
            Position = 2
            df.Position.loc[rownum] = Position
            df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1

        elif df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:    # FE_FL < c and AE_FL > a ---> No action, ride trend
            Counter1 += 1
            df.Position.loc[rownum] = df.Position.loc[rownum - 1]
            df.P_L.loc[rownum] = df.close.loc[rownum] - En_price1
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1     
    else:
        rownum += 1


IndentationError: expected an indented block (<ipython-input-32-def6b4183c4a>, line 7)

In [23]:
rownum = 1

while df.TrendDay.loc[rownum] >= 1:
    if df.open.loc[rownum] > df.close.loc[rownum]:
        candle = "dark"
    elif df.open.loc[rownum] < df.close.loc[rownum]:
        candle = "open"
    elif df.open.loc[rownum] == df.close.loc[rownum]:
        candle_range = df.high.loc[rownum] - df.low.loc[rownum]
        if df.open.loc[rownum] >= (df.low.loc[rownum] + candle_range/2):   # open & close occur at middle or in top half of range
            candle = "doji_open"
        else:
            candle = "doji_dark"    # open & close occur in lower half of range
                
                
                
                
                 # Case 2:  Position = 0: check for Re-Entry on two positions or ride zero
        
    if df.Position.loc[rownum - 1] == 0 and RE_CAN == True:
            
        """ RE_U: price at which to Re-enter on TWO positions """
        if df.NPP.loc[rownum - 1] <= re_entry["L"]["1"]:
            d = df.FL.loc[rownum - 1] + 1.2 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 30%N above FL
        elif df.NPP.loc[rownum - 1] <= re_entry["L"]["2"]:
            d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
        elif df.NPP.loc[rownum - 1] <= re_entry["L"]["3"]:
            d = df.FL.loc[rownum - 1] - 2 * X * df.N.loc[rownum - 1]     # Re-enter when price retraces below 50%N below FL
        else:
            d = df.FL.loc[rownum - 1] - 2.25 * X * df.N.loc[rownum - 1]                        

        df.d.loc[rownum] = d

        if df.low.loc[rownum] <= d:
            En_price1 = d - tick                        
            En_price2 = d - tick
            df.En_price1.loc[rownum] = En_price1
            df.En_price2.loc[rownum] = En_price2                        
            df.Counter1.loc[rownum] = 1
            df.Counter2.loc[rownum] = 1
            Position = 2
            df.Position.loc[rownum] = 2
            RE_CAN = False                              # toggle RE_CAN back to False
            df.RE_CAN.loc[rownum] = RE_CAN
            df.P_L.loc[rownum] = ((df.close.loc[rownum] - En_price1 + df.close.loc[rownum] - En_price2))                    
            rownum += 1
            continue
        else:
            Position = 0
            df.Position.loc[rownum] = 0
            RE_CAN = True
            df.RE_CAN.loc[rownum] = True 
     
     
     
     
     


In [ ]:
         
"""            
        # Case 3:  Position = 1, check if daily volatility exceeds SL or TP  
        if Position == 1 and df.high.loc[rownum] <=c and df.low.loc[rownum] >= a:
            df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
            df.Position.loc[rownum] = 1
            df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) 

"""

In [ ]:




                """ RE_U: price at which to Re-enter on TWO positions """
                if df.NPP.loc[rownum - 1] <= re_entry["L"]["1"]:
                    d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
                elif df.NPP.loc[rownum - 1] <= re_entry["L"]["2"]:
                    d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
                elif df.NPP.loc[rownum - 1] <= re_entry["L"]["3"]:
                    d = df.FL.loc[rownum - 1] - 2 * X * df.N.loc[rownum - 1]     # Re-enter when price retraces below 50%N below FL
                else:
                    d = df.FL.loc[rownum - 1] - 2.25 * X * df.N.loc[rownum - 1]                        

                df.d.loc[rownum] = d

                if df.low.loc[rownum] <= d:
                    En_price1 = d - tick                        
                    En_price2 = d - tick
                    df.En_price1.loc[rownum] = En_price1
                    df.En_price2.loc[rownum] = En_price2                        
                    df.Counter1.loc[rownum] = 1
                    df.Counter2.loc[rownum] = 1
                    Position = 2
                    df.Position.loc[rownum] = 2
                    RE_CAN = False                              # toggle RE_CAN back to False
                    df.RE_CAN.loc[rownum] = RE_CAN
                    df.P_L.loc[rownum] = ((df.close.loc[rownum] - En_price1 + df.close.loc[rownum] - En_price2))                    
                    rownum += 1
                    continue
                else:
                    Position = 0
                    df.Position.loc[rownum] = 0
                    RE_CAN = True
                    df.RE_CAN.loc[rownum] = True 


In [ ]:
rownum = 10
init_cap = float(25000)
while df.TrendDay.loc[rownum] >= 1:
                
    """  C A S E S   Here:  Position = , High/Low reaction to A/B/C/D,  RE_CAN status     """
                
    # case 1:  ONE position - RM functions do not impact initial position so carry to next day
    if Position == 1 and df.high.loc[rownum] <=c and df.low.loc[rownum] >= a:
        df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
        df.Position.loc[rownum] = 1
        df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) 
                    
        """ this conditional checks for same day Trend Change """
        if df.TrendDay.loc[rownum + 1] == 0:
            Counter1 = ""
            df.Counter1.loc[rownum + 1] = ""
            Position = 0
            df.Position.loc[rownum + 1] = ""
            Ex_price5 = df.close.loc[rownum]
            df.Ex_price5.loc[rownum] = Ex_price5
            df.P_L.loc[rownum] = En_price1 - Ex_price5
        else:
            rownum += 1
            continue                        
            
            
            

In [ ]:
type(init_cap)

In [ ]:


addpos = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},                            # syntax to call, enter script as follows:
          "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}                             # addpos["L"]["2"]  yields: 0.015
          }                                                                        # stoploss["S"]["3"]  yields: 0.0195

stoploss = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

takeprof = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

re_entry = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

X = 0.25   # General Risk Mgmt. multiplier

tick = 0.25
tick = float(tick)
tick_val = 12.50
tick_val = float(tick_val)


for rownum in range(1, 1500):
    Position = 0
    if df.TrendDay.loc[rownum] == 0 and df.TrendDir.loc[rownum] == "Up":
        SP = df.high.loc[rownum]                            # assign the 'high' slice to the SP variable
        df.SP.loc[rownum] = SP                              # equate the SP slice with the SP variable
        rownum += 1

        while df.TrendDay.loc[rownum] >= 1 and Position == 0:    # this 'while' loop finds initial entry, records initial P_L, then advances to next rownum

            if df.high.loc[rownum] > SP: 
                En_price1 = SP + tick
                df.En_price1.loc[rownum] = En_price1
                df.Counter1.loc[rownum] = 1                  # initialize Counter1
                Position = 1
                df.Position.loc[rownum] = 1                  # 'position' variable
                # df.Position.loc[rownum] = Position         # this [rownum] dataframe slice equal to this [rownum] value for position variable
                df.P_L.loc[rownum] = ( (df.close.loc[rownum] - En_price1)) 
                rownum += 1
                break

            else:
                SP = df.high.loc[rownum]
                df.SP.loc[rownum] = SP
                rownum += 1                

        while df.TrendDay.loc[rownum] > 1:

            """ AP_U function: AE_FL: price at which to Add to initial position """
            if df.NPP.loc[rownum - 1] < addpos["L"]["1"]:
                a = df.FL.loc[rownum - 1] +  X * df.N.loc[rownum - 1]
            elif df.NPP.loc[rownum - 1] <= addpos["L"]["2"]:                    
                a = df.FL.loc[rownum - 1]
            elif df.NPP.loc[rownum - 1] <= addpos["L"]["3"]:
                a = df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]
            else:
                a = df.FL.loc[rownum - 1] - 6.4 * X * df.N.loc[rownum - 1]
                
            df.a.loc[rownum] = a   # print value in dataframe slice 

            """ SL_U function: AE_FL: price at which to cut losses """
            if df.NPP.loc[rownum - 1] <= stoploss["L"]["1"]:
                b =  df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]     # stop loss when price retraces 1N below FL
            elif df.NPP.loc[rownum - 1] <= stoploss["L"]["2"]:
                b = df.FL.loc[rownum - 1] - 6 * X * df.N.loc[rownum - 1]  # stop loss when price retraces 150%N below FL
            elif df.NPP.loc[rownum - 1] <= stoploss["L"]["3"]:
                b = df.FL.loc[rownum - 1] - 7 * X * df.N.loc[rownum - 1]   # stop loss when price retraces  175%N below FL
            else:
                b =  df.FL.loc[rownum - 1] - 8 * X * df.N.loc[rownum - 1]

            df.b.loc[rownum] = b

            """ TP_U function: FE_FL: price at which to take profits """
            if df.NPP.loc[rownum - 1] <= takeprof["L"]["1"]:
                c = df.FL.loc[rownum - 1] + 5 * X * df.N.loc[rownum - 1]   # for NPP < 1.10%: take profit when price exceeds 125%N above previous FL
            elif df.NPP.loc[rownum - 1] <= takeprof["L"]["2"]:
                c = df.FL.loc[rownum - 1] + 5.6 * X * df.N.loc[rownum - 1]    # for NPP < 1.50%: take profit when price exceeds 140%N (5.6 * X) above previous FL
            elif df.NPP.loc[rownum - 1] <= takeprof["L"]["3"]:
                c = df.FL.loc[rownum - 1] + 7 * X * df.N.loc[rownum - 1]    # for NPP < 1.95%: take profit when price exceeds 175%N (7*X*N) above previous FL
            else:
                c = df.FL.loc[rownum - 1] + 7.5 * X * df.N.loc[rownum - 1]                    

            df.c.loc[rownum] = c

            # case 1:  ONE position - RM functions do not impact initial position so carry to next day
            if Position == 1 and df.high.loc[rownum] <=c and df.low.loc[rownum] >= a:
                df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
                df.Position.loc[rownum] = 1
                df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) 
                    
                """ this conditional checks for same day Trend Change """
                if df.TrendDay.loc[rownum + 1] == 0:
                    Counter1 = ""
                    df.Counter1.loc[rownum + 1] = ""
                    Position = 0
                    df.Position.loc[rownum + 1] = ""
                    Ex_price5 = df.close.loc[rownum]
                    df.Ex_price5.loc[rownum] = Ex_price5
                    df.P_L.loc[rownum] = En_price1 - Ex_price5
                else:
                    rownum += 1
                    continue            

            # case 2:  ONE position - FE_FL take profit on initial position   
            elif Position == 1 and df.high.loc[rownum] > c:
                Ex_price1 = c + tick
                df.Ex_price1.loc[rownum] = Ex_price1
                df.Counter1.loc[rownum] = 0
                Position = 0
                df.Position.loc[rownum] = 0
                RE_CAN = True
                df.RE_CAN.loc[rownum] = RE_CAN
                df.P_L.loc[rownum] = (Ex_price1 - En_price1)

                """ this conditional checks for same day Trend Change """
                if df.TrendDay.loc[rownum + 1] == 0:
                    Counter1 = ""
                    df.Counter1.loc[rownum + 1] = ""
                    Position = 0
                    df.Position.loc[rownum + 1] = ""
                    Ex_price5 = df.close.loc[rownum]
                    df.Ex_price5.loc[rownum] = Ex_price5
                    df.P_L.loc[rownum] = En_price1 - Ex_price5
                else:
                    rownum += 1
                    continue                    

            # case 3:  FLAT due to FE_FL triggering TP - look for AE_FL retracement to Re-enter TWO positions
            elif Position == 0 and df.RE_CAN.loc[rownum] == True:

                """ RE_U: price at which to Re-enter on TWO positions """
                if df.NPP.loc[rownum - 1] <= re_entry["L"]["1"]:
                    d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
                elif df.NPP.loc[rownum - 1] <= re_entry["L"]["2"]:
                    d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
                elif df.NPP.loc[rownum - 1] <= re_entry["L"]["3"]:
                    d = df.FL.loc[rownum - 1] - 2 * X * df.N.loc[rownum - 1]     # Re-enter when price retraces below 50%N below FL
                else:
                    d = df.FL.loc[rownum - 1] - 2.25 * X * df.N.loc[rownum - 1]                        

                df.d.loc[rownum] = d

                if df.low.loc[rownum] <= d:
                    En_price1 = d - tick                        
                    En_price2 = d - tick
                    df.En_price1.loc[rownum] = En_price1
                    df.En_price2.loc[rownum] = En_price2                        
                    df.Counter1.loc[rownum] = 1
                    df.Counter2.loc[rownum] = 1
                    Position = 2
                    df.Position.loc[rownum] = 2
                    RE_CAN = False                              # toggle RE_CAN back to False
                    df.RE_CAN.loc[rownum] = RE_CAN
                    df.P_L.loc[rownum] = ((df.close.loc[rownum] - En_price1 + df.close.loc[rownum] - En_price2))                    
                    rownum += 1
                    continue
                else:
                    Position = 0
                    df.Position.loc[rownum] = 0
                    RE_CAN = True
                    df.RE_CAN.loc[rownum] = True 

                    """ this conditional checks for same day Trend Change """
                    if df.TrendDay.loc[rownum + 1] == 0:
                        Counter1 = ""
                        df.Counter1.loc[rownum + 1] = ""
                        Position = 0
                        df.Position.loc[rownum + 1] = ""
                        Ex_price5 = df.close.loc[rownum]
                        df.Ex_price5.loc[rownum] = Ex_price5
                        df.P_L.loc[rownum] = En_price1 - Ex_price5
                    else:
                        rownum += 1
                        continue             

            # case 4:  TWO positions - FE_FL take profit on two positions
            elif Position == 2 and df.high.loc[rownum] > c:
                Ex_price1 = c + tick
                Ex_price2 = c + tick
                df.Counter1.loc[rownum] = 0
                df.Counter2.loc[rownum] = 0
                Position = 0
                df.Position.loc[rownum] = 0
                RE_CAN = True
                df.RE_CAN.loc[rownum] = RE_CAN
                df.P_L.loc[rownum] =  ( ((Ex_price1 - En_price1) + (Ex_price2 - En_price2)) ) 
                rownum += 1
                continue

            # case 5: TWO positions - RM functions do no impact position with contracts so carry to next rownum
            elif Position == 2 and df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:
                df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
                df.Counter2.loc[rownum] = df.Counter2.loc[rownum-1] + 1
                Position = 2
                df.Position.loc[rownum] = 2
                df.P_L.loc[rownum] = (((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2))) 

                """ this conditional checks for same day Trend Change """
                if df.TrendDay.loc[rownum + 1] == 0:
                    Counter1 = ""
                    df.Counter1.loc[rownum + 1] = ""
                    Position = 0
                    df.Position.loc[rownum + 1] = ""
                    Ex_price5 = df.close.loc[rownum]
                    df.Ex_price5.loc[rownum] = Ex_price5
                    df.P_L.loc[rownum] = En_price1 - Ex_price5
                else:
                    rownum += 1
                    continue                    


            # case 6a: ONE position - AE_FL - add to initial position and STOP out of TWO positions - look for next trend start
            elif Position == 1 and df.low.loc[rownum] < a:
                En_price2 = a - tick
                df.En_price2.loc[rownum] = En_price2
                df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
                df.Counter2.loc[rownum] = 1
                Position = 2
                df.Position.loc[rownum] = 2
                df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2) 


                # case 6b: TWO positions - AE_FL - STOP OUT both positions and increment until next TrendDay=0
                if df.low.loc[rownum] < b:
                    Ex_price3 = b - tick
                    Ex_price4 = b - tick
                    df.Ex_price3.loc[rownum] = Ex_price3
                    df.Ex_price4.loc[rownum] = Ex_price4               
                    df.Counter1.loc[rownum] = 0
                    df.Counter2.loc[rownum] = 0
                    Position = 0
                    df.Position.loc[rownum] = 0 
                    df.P_L.loc[rownum] = (En_price1 - Ex_price3) + (En_price2 - Ex_price4) 
                    rownum += 1
                    while df.TrendDay.loc[rownum] != 0:
                        rownum += 1
                        continue
                    break
                else:
                    rownum += 1
                    continue
        else:
            rownum += 1
    else:
        rownum += 1        

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
